<a href="https://colab.research.google.com/github/gombergere/arthomas/blob/master/YXL0916.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
#Appli de réference
#améliorée au fil du temps
#noter la bibli cython=C+Python-> performances égales au C

In [0]:

!pip install pydrive
!git clone https://github.com/thtrieu/darkflow.git 
!pip install Cython
!pip install pytube
!pip install PyDrive


In [0]:
import os 
os.chdir('/content/darkflow') 
!pwd

In [0]:
!pip install -e .

In [0]:
import pydrive
import time
from tqdm import tqdm
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from google.colab import files
from oauth2client.client import GoogleCredentials

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [0]:
#repare un oubli de l'auteur
!mkdir /content/darkflow/bin

In [0]:
download = drive.CreateFile({'id': '1J-GZ6mvhu12j19zZtq9fTbnyJeCgxE0g'}) # insert the fild ID here
download.GetContentFile('/content/darkflow/bin/yolov2.weights')

In [0]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np

from darkflow.net.build import TFNet
import cv2

In [0]:
options = {
    'model': 'cfg/yolo.cfg',
    'load': 'bin/yolov2.weights',
    'threshold': 0.3,
    'gpu' : 1.0
          }
tfnet = TFNet(options)

In [0]:
from google.colab import files
file = files.upload()

In [0]:
import pprint as pp

original_img = cv2.imread("multiple.jpg")
original_img = cv2.cvtColor(original_img, cv2.COLOR_BGR2RGB)

# sauf erreur l'instruction suivante est le "moteur" de la détection
results = tfnet.return_predict(original_img)
#results est entré ensite commme donnée de boxing qui fabrique l'image détectée
# l'instruction suivante édite le résultat de la détection.Voir le listing suivant

pp.pprint(results)

In [0]:
fig, ax = plt.subplots(figsize=(10, 10))
ax.imshow(original_img)

In [0]:
def boxing(original_img, predictions):
    newImage = np.copy(original_img)

    for result in predictions:
        top_x = result['topleft']['x']
        top_y = result['topleft']['y']

        btm_x = result['bottomright']['x']
        btm_y = result['bottomright']['y']

        confidence = result['confidence']
        label = result['label'] + " " + str(round(confidence, 3))

        if confidence > 0.3:
            newImage = cv2.rectangle(newImage, (top_x, top_y), (btm_x, btm_y), (255,0,0), 3)
            newImage = cv2.putText(newImage, label, (top_x, top_y-5), cv2.FONT_HERSHEY_COMPLEX_SMALL , 0.8, (0, 230, 0), 1, cv2.LINE_AA)
            
    return newImage

In [0]:
fig, ax = plt.subplots(figsize=(20, 10))
ax.imshow(boxing(original_img, results))

In [0]:
#chargement dde la video
from google.colab import files
file = files.upload()

In [0]:
cap = cv2.VideoCapture("video.mp4")
width = cap.get(cv2.CAP_PROP_FRAME_WIDTH)   
height = cap.get(cv2.CAP_PROP_FRAME_HEIGHT) 
from google.colab.patches import cv2_imshow

fourcc = cv2.VideoWriter_fourcc(*'DIVX')
out = cv2.VideoWriter('output.avi',fourcc, 20.0, (int(width), int(height)))

while(True):
   # Capture frame-by-frame
   ret, frame = cap.read()
    
   if ret == True:
       frame = np.asarray(frame)
       results = tfnet.return_predict(frame)

       new_frame = boxing(frame, results)

       # Display the resulting frame
       out.write(new_frame)
        
       #cv2.imshow('frame',new_frame)
       #cv2_imshow('frame')
       if cv2.waitKey(1) & 0xFF == ord('q'):
            break
   else:
       break

# When everything done, release the capture
cap.release()
out.release()
cv2.destroyAllWindows()

# et voilà ,c'est fait...